 # 毕业设计

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (RandomizedSearchCV, KFold,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


 ## 获取数据

In [ ]:
path = '../../../Dataset/Stock/'
freq_dict = {
    'day': 'B',
    'week': '7D',
    'month': 'BM'
}
offset_dict = {
    'day': pd.offsets.BDay(),
    'week': pd.offsets.Week(),
    'month': pd.offsets.BusinessMonthEnd()
}


def get_data(frequency: str, T: str) -> None:
    data_path = os.path.join(path, frequency)
    data_1 = pd.read_csv('Data/test.csv')  # 空文件，迭代用
    freq = freq_dict[frequency]
    offset = offset_dict[frequency]

    for file in os.listdir(data_path):
        df = pd.read_csv(os.path.join(data_path, file), index_col='index', parse_dates=True)  # 以时间序列为索引
        df.dropna(inplace=True)  # 去除空值

        up = df.query('open<close')  # 阳线
        down = df.query('open>close')  # 阴线
        up = up.tshift(1, freq=freq)  # 将阳线时间戳后移

        new = up.join(down, how='inner', lsuffix='_up', rsuffix='_down')  # 合并
        new.query('vol_down>vol_up', inplace=True)  # vol(2)>vol(1)

        # close(2)>close(1)
        first = new.query('close_down>close_up')
        low_mean = pd.Series(index=first.index, name='T_mean')
        for start in first.index:
            period = pd.date_range(start=start + offset, periods=int(T), freq=freq)
            # 时间范围可能无效
            try:
                low_mean[start] = df.loc[period].low.mean()
            except:
                first.drop(index=start, inplace=True)  # 清除无效行
                continue
        low_mean = pd.Series(low_mean, index=first.index, name='T_mean')
        piece = pd.concat([first, low_mean], axis=1, join='inner').reset_index(drop=True)  # 去除时间索引
        data_1 = pd.concat([data_1, piece])

    # close(2)>close(1) & low(2)<low(1)
    data_2 = data_1.query('low_down<low_up')

    data_1.to_csv(os.path.join('Data', 'T=' + T, frequency + '_1.csv'), index=False)
    data_2.to_csv(os.path.join('Data', 'T=' + T, frequency + '_2.csv'), index=False)



In [ ]:
for T in np.arange(2, 16).astype(np.str):
    for frequency in ['day', 'week']:  # month数据量太少，无意义
        get_data(frequency, T)


 ## 构建模型

In [ ]:
path = 'Data/'


In [ ]:


def del_outliers(df: pd.DataFrame) -> pd.DataFrame:
    q1 = df['T_mean'].quantile(0.25)
    q3 = df['T_mean'].quantile(0.75)
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr
    return df[(df['T_mean'] > low) & (df['T_mean'] < high)]


In [ ]:


def training_with_all(state: str, T_range: np.ndarray, param: dict) -> pd.DataFrame:
    result = pd.DataFrame(columns=['mse', 'gamma', 'C'], index=T_range)

    print(state, ' begin.')

    for T in T_range:
        data_path = os.path.join(path, 'T=' + T, state + '.csv')
        df = pd.read_csv(data_path)

        new = del_outliers(df)

        X = new.drop(columns='T_mean')
        y = new['T_mean']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        std_scaler = StandardScaler()
        X_train = std_scaler.fit_transform(X_train)
        X_test = std_scaler.transform(X_test)

        cv = KFold(n_splits=5, random_state=42)
        randomcv = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=param, scoring='neg_mean_squared_error', n_jobs=-1, cv=cv, random_state=42, verbose=2)
        randomcv.fit(X_train, y_train)

        y_pred = randomcv.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        result['mse'][T] = mse
        result['gamma'][T] = randomcv.best_estimator_.gamma
        result['C'][T] = randomcv.best_estimator_.C
        print('T=' + T + ': complete.', 'mse=', mse, ', param=', randomcv.best_params_)

    print(state, ' complete.')

    return result



In [ ]:
T_range = np.arange(2, 16).astype(np.str)
param = {
    'gamma': np.logspace(-4, -1, 4),
    'C': np.logspace(1, 4, 4)
}
for state in ['day_1', 'day_2', 'week_1', 'week_2']:
    result = training_with_all(state=state, T_range=T_range, param=param)
    result.to_csv('result_' + state + '.csv', index_label='T')


 ### 测试

In [ ]:
T_range = np.arange(2, 16).astype(np.str)
param = {
    'gamma': np.logspace(-4, -1, 4),
    'C': np.logspace(1, 4, 4)
}
test = training_with_all(state='day_1', T_range=T_range, param=param)
test.to_csv('result_' + state + '.csv', index_label='T')


 ## 结论

### 重置数据

In [ ]:


def reset_data(state: str) -> None:
    df = pd.read_csv('result_' + state + '.csv', index_col='T')  # type:pd.DataFrame
    df = df.values.tolist()
    for i in np.arange(14):
        tmp = eval(df[i][1])
        df[i].pop()
        df[i] += [tmp['gamma'], tmp['C']]
    df = pd.DataFrame(df, index=np.arange(2, 16), columns=['mse', 'gamma', 'C'])
    df.to_csv('result_' + state + '.csv', index_label='T')



In [ ]:
for state in ['day_1', 'day_2', 'week_1', 'week_2']:
    reset_data(state)


### 图表

In [2]:
day_1 = pd.read_csv('result_day_1.csv', index_col='T')
day_2 = pd.read_csv('result_day_2.csv', index_col='T')
week_1 = pd.read_csv('result_week_1.csv', index_col='T')
week_2 = pd.read_csv('result_week_2.csv', index_col='T')


In [4]:
fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type': 'scene'},
            {'type': 'scene'}
            ],
            [
            {'type': 'scene'},
            {'type': 'scene'}
            ]
    ],
    subplot_titles=['day_1', 'day_2', 'week_1', 'week_2']
)
fig.add_traces(
    [go.Scatter3d(
        x=df.index,
        y=df['gamma'],
        z=df['C'],
        mode='markers',
        marker_size=df['mse'] * 10,
        name=name,
        text=df['mse'],
        hovertemplate='T=%{x}<br>gamma=%{y}<br>C=%{z}<br>mse=%{text}',
    ) for df, name in [(day_1, 'day_1'), (day_2, 'day_2'), (week_1, 'week_1'), (week_2, 'week_2')]],
    rows=[1, 1, 2, 2],
    cols=[1, 2, 1, 2]
)
scene_dict=dict(
    xaxis_title='T',
    yaxis=dict(
        type='log',
        title='gamma',
        tickvals=np.logspace(-4, -1, 4)
    ),
    zaxis=dict(
        type='log',
        title='C',
        tickvals=np.logspace(1, 4, 4)
    ),
)
fig.update_layout(
    scene=scene_dict,
    scene2=scene_dict,
    scene3=scene_dict,
    scene4=scene_dict,
    showlegend=False,
    width=1000,
    height=1000
)
fig.show()